In [0]:
!pip install kaggle

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Configure Kaggle Credentials

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "samikshalm"
os.environ["KAGGLE_KEY"] = "e565feb6f0501f1f12f8eccf8bf49878"

print("Kaggle credentials configured!")

Kaggle credentials configured!


Create Database Schema

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

Create Volume for Data Storage

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

5. Download Dataset from Kaggle

In [0]:
!cd /Volumes/workspace/ecommerce/ecommerce_data
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
 11%|████▌                                   | 500M/4.29G [00:02<00:20, 196MB/s]

Connection error: OSError: [Errno 27] File too large
Retrying in 2.8 seconds... (attempt 1/5)
Retry 1/5: Resuming from 524288000 bytes (4082432907 bytes left)...
 11%|█████▎                                         | 500M/4.29G [00:00<?, ?B/s]

Connection error: FileNotFoundError: [Errno 2] No such file or directory: '/Workspace/Users/samikshalm@gmail.com/Drafts/ecommerce-behavior-data-from-multi-category-store.zip'
Retrying in 4.4 seconds... (attempt 2/5)
Retry 2/5: Resuming from 524288000 bytes (4082432907 bytes left)...
 11%|█████▎                                         | 500M/4.29G [00:00<?, ?B/s]

Connection error: FileNotFoundError: [Errno 2] No such file or directory: '/Workspace/Users/samikshalm@gmail.com/Drafts/ecommerce-behavior-data-from-multi-category-store.zip

In [0]:
!cd /Volumes/workspace/ecommerce/ecommerce_data
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
 11%|████▌                                   | 500M/4.29G [00:02<00:22, 185MB/s]

Connection error: OSError: [Errno 27] File too large
Retrying in 2.1 seconds... (attempt 1/5)
Retry 1/5: Resuming from 524288000 bytes (4082432907 bytes left)...
 11%|█████▎                                         | 500M/4.29G [00:00<?, ?B/s]

Connection error: FileNotFoundError: [Errno 2] No such file or directory: '/Workspace/Users/samikshalm@gmail.com/Drafts/ecommerce-behavior-data-from-multi-category-store.zip'
Retrying in 4.6 seconds... (attempt 2/5)
Retry 2/5: Resuming from 524288000 bytes (4082432907 bytes left)...
 11%|█████▎                                         | 500M/4.29G [00:00<?, ?B/s]

Connection error: FileNotFoundError: [Errno 2] No such file or directory: '/Workspace/Users/samikshalm@gmail.com/Drafts/ecommerce-behavior-data-from-multi-category-store.zip

In [0]:
!cd /Volumes/workspace/ecommerce/ecommerce_data
!rm -f ecommerce-behavior-data-from-multi-category-store.zip
!ls -lh

total 9.0K
-rwxrwxrwx 1 root root 8.7K Feb 20 11:48 'Untitled Notebook 2026-02-18 20:38:41.ipynb'


In [0]:
%restart_python

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

In [0]:
import os
print(os.listdir("/Volumes/workspace/ecommerce/ecommerce_data"))

['2019-Nov.csv', '2019-Oct.csv']


Display Dataset Statistics and Schema

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

October 2019 - Total Events: 103

SCHEMA:
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



Display Sample Data

In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-----------------------+----------+----------+-----------+-----------------------------------+--------+------+---------+------------------------------------+
|_c0                    |_c1       |_c2       |_c3        |_c4                                |_c5     |_c6   |_c7      |_c8                                 |
+-----------------------+----------+----------+-----------+-----------------------------------+--------+------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id|category_code                      |brand   |price |user_id  |user_session                        |
|2019-10-01 00:00:00 UTC|view      |44600062  |2.10381E+18|NULL                               |shiseido|35.79 |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2.05301E+18|appliances.environment.water_heater|aqua    |33.2  |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00

In [0]:
csv_path = "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"
delta_path = "/Volumes/workspace/ecommerce/ecommerce_data/oct_delta"

df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(csv_path))

(df.write
 .format("delta")
 .mode("overwrite")
 .save(delta_path))

In [0]:
df = spark.read.format("delta").load("/Volumes/workspace/ecommerce/ecommerce_data/oct_delta")
df.write.saveAsTable("workspace.ecommerce.oct_delta_managed")

In [0]:
df_new = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
df_new.write.format("delta").mode("append").saveAsTable("workspace.ecommerce.oct_delta_managed")

In [0]:
spark.sql("OPTIMIZE workspace.ecommerce.oct_delta_managed")
print("Delta table optimized.")

Delta table optimized.


In [0]:
import time

start = time.time()
df = spark.read.table("workspace.ecommerce.oct_delta_managed")
row_count = df.count()
end = time.time()
print(f"Row count: {row_count}")
print(f"Read time: {end - start:.2f} seconds")

Row count: 201
Read time: 0.60 seconds
